# Gemini intro

In [5]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Generate some funny jokes about data engineering. Give 5 points",
)
print(response.text)

Here are 5 funny jokes about data engineering:

1.  **What's a data engineer's favorite type of alarm clock?**
    One that goes off at 3 AM to tell them a data pipeline failed.

2.  **A data scientist walks up to a data engineer and asks, "Can you just pull me that data?"**
    The data engineer sighs, "There's no 'just' in data engineering."

3.  **Why was the data engineer always calm during a fire drill?**
    Because he knew nothing could be more catastrophic than an unexpected schema change.

4.  **What's a data engineer's favorite household chore?**
    Cleaning data. Because no matter how much you do, it'll never *really* be clean.

5.  **What's a data engineer's spirit animal?**
    A salmon, because they're constantly fighting upstream to get the data where it needs to go.


In [8]:
response.__dict__.keys()

dict_keys(['sdk_http_response', 'candidates', 'create_time', 'model_version', 'prompt_feedback', 'response_id', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])

## Analyze tokens

- basic units of text for LLMs
- can be short as one character or as long as one word

The free tier in gemini API allows for (Gemini 2.5 flash)

- Requesrs per minute (RPM): 10
- Tokens per minute (TPM): 250 000
- Requests per dat (RDP): 250

In [10]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=208,
  prompt_token_count=13,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  thoughts_token_count=1855,
  total_token_count=2076
)

In [12]:
metadata = response.usage_metadata
metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=208,
  prompt_token_count=13,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  thoughts_token_count=1855,
  total_token_count=2076
)

In [11]:
from pydantic import BaseModel

isinstance(response, BaseModel)

True

In [13]:

print("Output tokens - number of tokens in model response")
print(f"{metadata.candidates_token_count = }")

Output tokens - number of tokens in model response
metadata.candidates_token_count = 208


In [15]:
print("Tokens in user input or user prompt")
print(f"{metadata.prompt_token_count = }")

Tokens in user input or user prompt
metadata.prompt_token_count = 13


In [16]:
metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=208,
  prompt_token_count=13,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  thoughts_token_count=1855,
  total_token_count=2076
)

In [18]:
# a lot of tokens used here
print("Tokens used for internal thinking")
print(f"{metadata.thoughts_token_count = }")

Tokens used for internal thinking
metadata.thoughts_token_count = 1855


In [19]:
print("Total tokens used - this is the billing number")
print(f"{metadata.total_token_count = }")

Total tokens used - this is the billing number
metadata.total_token_count = 2076


## Thinking

In [21]:
from google.genai import types

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Generate some funny jokes about data engineering. Give 5 points",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)
print(response.text)

Here are 5 funny jokes about data engineering:

1.  Why did the data engineer break up with the API?
    Because there were too many **missing fields** in their relationship!

2.  What's a data engineer's favorite type of music?
    **Stream**ing! (Especially when it's real-time.)

3.  How many data engineers does it take to change a lightbulb?
    None. They just declare a new variable `is_light_on = True`, run a Spark job to process the state change, and then wait for someone to build a dashboard to visualize it.

4.  A data engineer walks into a bar. The bartender asks, "What can I get you?"
    The data engineer replies, "I'd like a beer, but can you make sure it's fully **normalized**, properly **partitioned**, and that I have read-only access to the source keg?"

5.  What do you call a data engineer who's always stressed?
    Under **pressure** (to deliver those pipelines on time!).


In [22]:
print(repr(response.usage_metadata))

print("Ah much cheaper, but is the result as good as the thinking?")

GenerateContentResponseUsageMetadata(
  candidates_token_count=232,
  prompt_token_count=13,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  total_token_count=245
)
Ah much cheaper, but is the result as good as the thinking?


## System instructions

In [25]:
system_instruction = """You are a joking robot called Ro Båt, which 
        will always answer with a programming joke.
        """

prompt = "What is the weather today?"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
    ),
    contents=prompt,
)

response.text

'It\'s the perfect weather to debug code outside. Just remember, sunny days are great for spotting errors, but dark nights are when the real "bugs" come out! 😉\n'

In [26]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=37,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=37
    ),
  ],
  prompt_token_count=30,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=30
    ),
  ],
  total_token_count=67
)

In [27]:
print(f"{len(system_instruction.split()) = }")
print(f"{len(prompt.split()) = }")
print("plus some formatting overhead")

print(f"prompt token count {response.usage_metadata.prompt_token_count}")

len(system_instruction.split()) = 16
len(prompt.split()) = 5
plus some formatting overhead
prompt token count 30


## Temperature

In [29]:
story_prompt = "write a 2 sentence story about a gray rabbit"

boring_story = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=0.0),
)

print(boring_story.text)

The gray rabbit twitched its nose, sensing the distant rumble of a lawnmower and immediately darted into the overgrown rose bushes, its fluffy tail disappearing in a flash of white. Safe within the thorny embrace, it nibbled on a fallen petal, the sweet scent masking the mechanical threat.



In [30]:
# you can see that the outputs are similar to the first when temperature is 0
boring_story = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=0.0),
)

print(boring_story.text)

The gray rabbit twitched its nose, sensing the distant rumble of a lawnmower and immediately darted into the overgrown rose bushes. Safe within the thorny embrace, it nibbled on a fallen petal, the sweet scent masking the fear that still lingered in its twitching whiskers.



In [31]:
creative_story = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=2.0),
)

print(creative_story.text)

Barnaby, a gray rabbit with a nervous twitch in his nose, hopped out of his burrow, sniffing the air cautiously. Today, however, the scent of sunshine and wildflowers was stronger than his fear, beckoning him toward an adventure beyond the familiar field.



In [32]:
# very different story
creative_story = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=2.0),
)

print(creative_story.text)

Barnaby, a rabbit of unassuming gray, discovered a patch of glowing blue mushrooms nestled beneath a twisted oak. One nibble turned the forest floor into a symphony of color and Barnaby into a vibrant shade of electric blue.



## Multimodal inputs

In [36]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents={
        "parts": [
            {"text": "Tell me about this dude, write in markdown format."},
            {
                "inline_data": {
                    "mime_type": "image/png",
                    "data": open("assets/kokchun.png", "rb").read(),
                }
            },
        ]
    },
)
print(response.text)

Okay, here's a description of the dude, based on the image:

*   **Appearance:** He's a young man with short, dark hair. He's wearing glasses with a square frame.

*   **Outfit:** He's wearing a black and white raglan shirt. The shirt is printed with two graphics:
    *   A bell curve labeled "NORMAL DISTRIBUTION"
    *   Below that, a sad ghost-like shape that represents "PARANORMAL DISTRIBUTION".

*   **Activity:** He's using a laptop with an Apple logo.

*   **Expression:** He looks focused and a bit serious, perhaps deep in thought or concentrating on something on the screen.

*   **Vibe:** He seems like a person who appreciates a good math joke or has an interest in statistics. He's possibly a student or someone working in a tech-related field.



In [37]:
from pathlib import Path

Path("exports").mkdir(exist_ok=True)

In [38]:
with open("exports/image_description.md", "w") as f:
    f.write(response.text)